In [4]:
import numpy as np
import pandas as pd
import subprocess
import os
import math
import bz2
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from PIL import Image
from datetime import datetime, timedelta


In [15]:
# 共通設定
shape = (3000, 3000)
west_japan = (slice(550, 750), slice(1000, 1400))
kochi_area = (slice(625, 675), slice(1150, 1250))

# 日付と時刻
date = "20250904"
time = "0000" #UTC(0000は日本の09:00)

# ひまわりデータ
url = f"ftp://hmwr829gr.cr.chiba-u.ac.jp/gridded/FD/V20190123/202509/4KM/{date}/"
band_list = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B13']
name_list = ['vis.01', 'vis.02', 'ext.01', 'vis.03', 'sir.01', 'sir.02', 'tir.01']
wave_list = ['0.47μm', '0.51μm', '0.64μm', '0.86μm', '1.6μm', '2.3μm', '10.4μm']

# 雲高さ推定
h = 6.626e-34           # プランク定数 [J·s]
c = 3.0e8               # 光速 [m/s]
k = 1.381e-23           # ボルツマン定数 [J/K]
lambda_m = 10.4e-6      # 波長 [m]
T_surface = 305.0       # 地表温度 [K]
lapse_rate = 6.5        # 気温減率 [K/km]

# 雲厚さ推定
rho_w = 1000            # 水の密度 [kg/m³]
r_eff = 10e-6           # 雲粒の有効半径 [m]
LWC = 0.0005            # 液体水量 [kg/m³]

# データ読み込み関数
def load_bz2(fname, shape=shape):
    with bz2.BZ2File(fname) as f:
        return np.frombuffer(f.read(), '>f4').reshape(shape)

# コントラスト補正、ガンマ補正関数
def correct_image(image):
    vmin = np.percentile(image , 10)
    vmax = np.percentile(image , 90)
    image  = np.clip((image - vmin) / (vmax - vmin + 1e-6), 0, 1)
    image = image ** 0.5 #gammma値
    return(image)

# 放射輝度（撮影された光強度）→輝度温度（物体温度）の変換（プランクの法則）
def Brightness_Temperature(image):
    image = image * 1e6
    term1 = (2 * h * c**2) / (lambda_m**5)
    term2 = term1 / image
    T = (h * c) / (lambda_m * k * np.log(term2 + 1))
    return T

# 輝度温度→高度に変換（1km上がるごとにlapse_rate下がる）
def Cloud_Top(T):
    return (T_surface - T) / lapse_rate

# 輝度温度から光学的厚さtau、物理的厚さthickに変換
def Thick(T):
    tau = 0.5 * T + 1
    thick = (2 * rho_w * r_eff * tau) / (3 * LWC) /1000
    return thick


In [16]:
#ファイルダウンロード
file_list = {}

# ファイルをダウンロード
for i in range(len(band_list)):
    file_list[i] = f"{date}{time}.{name_list[i]}.rad.fld.4km.bin.bz2"
    subprocess.run(["curl", "-O", url + file_list[i]], check=True)

In [ ]:
# 生データの表示
fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes_flat = axes.ravel()

for i in range(6):
    ax = axes_flat[i]
    image = correct_image(load_bz2(file_list[i]))
    ax.imshow(image, cmap='gray')
    ax.set_title(band_list[i])
    ax.axis('off')

plt.show()


In [ ]:
# バンド1,2,3を結合
r = correct_image(load_bz2(file_list[band_list.index('B03')]))
g = correct_image(load_bz2(file_list[band_list.index('B02')]))
b = correct_image(load_bz2(file_list[band_list.index('B01')]))
image = np.stack([r, g, b], axis=-1)

plt.imshow(image)
plt.show()

In [ ]:
# 水分分布を可視化
r = correct_image(load_bz2(file_list[band_list.index('B05')]))
g = correct_image(load_bz2(file_list[band_list.index('B04')]))
b = correct_image(load_bz2(file_list[band_list.index('B03')]))
image = np.stack([r, g, b], axis=-1)

plt.imshow(image)
plt.show()

In [ ]:
# 指定エリアのみ表示
plt.imshow(image[west_japan])
plt.show()

In [ ]:
# 雲の検出
threshold=0.35

image = correct_image(load_bz2(file_list[band_list.index('B03')]))
#image = np.array(image)
image[image>threshold] = 1
image[image<=threshold] = 0

plt.imshow(image[west_japan], cmap='gray')
plt.show()

In [ ]:
# 雲の高さを計算
image = load_bz2(file_list[band_list.index('B13')])
image = image[west_japan] #処理が重いため地域を限定すること。
height = Cloud_Top(Brightness_Temperature(image))

Z = height
x = np.arange(Z.shape[1])
y = np.arange(Z.shape[0])
X, Y = np.meshgrid(x, y)

fig = go.Figure(
    data=[go.Surface(z=np.flipud(Z), x=X, y=Y)
    ])
fig.update_layout(
    title = "雲高さ",
    scene = dict(aspectratio=dict(x=2, y=1, z=0.5))
    )
fig.show()


In [ ]:
# 雲の厚さを計算
image = load_bz2(file_list[band_list.index('B06')])
image = image[west_japan] #処理が重いため地域を限定すること。
thickness = Thick(Brightness_Temperature(image))

Z = thickness
x = np.arange(Z.shape[1])
y = np.arange(Z.shape[0])
X, Y = np.meshgrid(x, y)

fig = go.Figure(
    data=[go.Surface(z=np.flipud(Z), x=X, y=Y)
    ])
fig.update_layout(
    title = "雲厚さ",
    scene = dict(aspectratio=dict(x=2, y=1, z=0.5))
    )
fig.show()

In [ ]:
# 雲の上と下をプロット
base = height - thickness
mask = (height > 2) & (thickness > 0.1)
Z1 = np.where(mask, height, np.nan)
Z2 = np.where(mask, base, np.nan)
size_array = np.abs(thickness.ravel()) * 2
size_array = np.nan_to_num(size_array, nan=1)

x = np.arange(height.shape[1])
y = np.arange(height.shape[0])
X, Y = np.meshgrid(x, y)

fig = go.Figure()
fig.add_trace(
    go.Scatter3d(
        x = X.ravel(),
        y = Y.ravel(),
        z = np.flipud(Z1).ravel(),
        mode = 'markers',
        marker = dict(size=size_array, color='blue', opacity=0.5),
        showlegend=False
        )
    )
fig.add_trace(
    go.Scatter3d(
        x = X.ravel(),
        y = Y.ravel(),
        z = np.flipud(Z2).ravel(),
        mode = 'markers',
        marker = dict(size=size_array, color='black', opacity=0.5),
        showlegend=False
        )
    )
fig.update_layout(
    scene = dict(aspectratio=dict(x=2, y=1, z=0.5))
    )

x0, y0, z0 = 200, 100, 0  # 視点（地表中心）
angle_deg = 80 # 視野角
num_layers = 50
num_points = 50
h = 20
theta = np.linspace(0, 2 * np.pi, num_points)

# 各層の円を描画
for i in range(num_layers):
    z = z0 + h * (i / num_layers)
    r = z * np.tan(np.radians(angle_deg))
    x = x0 + r * np.cos(theta)
    y = y0 + r * np.sin(theta)
    fig.add_trace(go.Scatter3d(
        x=x, y=y, z=np.full_like(x, z),
        mode='lines',
        line=dict(color='green', width=5),
        showlegend=False
    ))

fig.show()

In [ ]:
# 地上から見上げた雲分布

# 雲天ベクトルの地表面への投影
x_size, y_size = 1200, 600
dx = (X - x0) * (x_size / (height.shape[1] - 1))
dy = (Y - y0) * (y_size / (height.shape[0] - 1))
u, v = dx / (z0 - Z1 + 1e-6), dy / (z0 - Z2 + 1e-6)
u2, v2 = dx / (z0 - Z2 + 1e-6), dy / (z0 - Z2 + 1e-6)

# 描画
fig, ax = plt.subplots(figsize=(6,6))
ax.scatter(u, v, color='blue', s=10)
ax.scatter(u2, v2, color='gray', s=2)

for angle in [1, 2, 5]:
    r = np.cos(np.radians(angle))/np.sin(np.radians(angle))
    ax.add_patch(plt.Circle((0, 0), r, fill=False))

fig.text(0.5, 0.12, 'South 1°', ha='center', va='bottom', fontsize=12, color="red")
ax.set_xlim(-60, 60)
ax.set_ylim(-60, 60)
plt.show()

In [ ]:
# 日射量計算（数学モデル）

# 地点情報（高知市）、太陽定数
lat, lon = 33.56, 133.53
std_meridian = 135.0
dni_extra = 1367

# JST前提の日時リスト（2025年8月・1時間ごと）
times = [datetime(2025, 8, 1) + timedelta(hours=i) for i in range(31 * 24)]

# 太陽高度角（経度補正あり）
def solar_elevation(t):
    doy = t.timetuple().tm_yday
    hour = t.hour + (std_meridian - lon) / 15.0
    decl = 23.45 * np.sin(np.deg2rad(360 * (284 + doy) / 365))
    omega = 15 * (hour - 12)
    elev = np.arcsin(
        np.sin(np.deg2rad(lat)) * np.sin(np.deg2rad(decl)) +
        np.cos(np.deg2rad(lat)) * np.cos(np.deg2rad(decl)) * np.cos(np.deg2rad(omega))
    )
    return max(np.rad2deg(elev), 0)

# 晴天時日射量（GHI）
def clear_sky(elev):
    zenith = np.radians(90 - elev)
    dni = dni_extra * np.exp(-0.1 * zenith)
    ghi = dni * np.cos(zenith)
    return ghi

# 計算とプロット
ghi_values = [clear_sky(solar_elevation(t)) for t in times]
plt.figure(figsize=(10, 4))
plt.plot(ghi_values, label='Clear-sky GHI', color='blue', linewidth=2)
plt.show()
